# Reproduce The DR16 paper result

Get Picca with different versions:
master: 
git clone https://github.com/igmhub/picca.git
export PICCA_BASE=<path to your picca>
pip install -r requirements.txt --user
python setup.py install --user

DR16 paper version:
git checkout tags/v4 -b publish



In [ ]:
Setup variable pointing to the DR16 spectra catalog:
export EBOSS_v5_13_0='/global/cfs/cdirs/cosmo/data/sdss/dr16/eboss/spectro/redux/v5_13_0/'

Setup another variable pointing to the DR16 quasar catalog:
export DR16='/global/cfs/cdirs/desi/science/lya/eboss_dr16/ting_reproduce_dr16'

export Reproduce_dr16=<path to your reproduction dr16>

In [ ]:
cp -r $DR16/Catalogs $Reproduce_dr16/Catalogs
cp -r $DR16/Scripts $Reproduce_dr16/Scripts
cd $Reproduce_dr16
mkdir Delta_calibration Delta_calibration/Delta Delta_calibration/Log/
mkdir Delta_calibration2 Delta_calibration2/Delta Delta_calibration2/Log/
mkdir Delta_LYA Delta_LYA/Delta Delta_LYA/Log/
mkdir Delta_LYB Delta_LYB/Delta Delta_LYB/Log/
mkdir Fits Correlations

In [ ]:
$PICCA_BASE/bin/picca_deltas.py --in-dir $EBOSS_v5_13_0/ --drq $Reproduce_dr16/Catalogs/cat_for_clustering.fits.gz --out-dir $Reproduce_dr16/Delta/Delta_calibration/Delta/ --iter-out-prefix $Reproduce_dr16/Delta/Delta_calibration/Log/delta_attributes --log $Reproduce_dr16/Delta/Delta_calibration/Log/input.log --dust-map $Reproduce_dr16/Catalogs/cat_for_clustering.fits.gz --rebin 3 --mode spplate --lambda-min 3600.0 --lambda-max 7235.0 --lambda-rest-min 2900.0 --lambda-rest-max 3120.0 --nit 20 --bi-max 0. --npix-min 50

In [ ]:
$PICCA_BASE/bin/picca_deltas.py --in-dir $EBOSS_v5_13_0/ --drq $Reproduce_dr16/Catalogs/cat_for_clustering.fits.gz --out-dir $Reproduce_dr16/Delta/Delta_calibration2/Delta/ --iter-out-prefix $Reproduce_dr16/Delta/Delta_calibration2/Log/delta_attributes --log $Reproduce_dr16/Delta/Delta_calibration2/Log/input.log --dust-map $Reproduce_dr16/Catalogs/cat_for_clustering.fits.gz --flux-calib $Reproduce_dr16/Delta/Delta_calibration/Log/delta_attributes.fits.gz --mask-file $Reproduce_dr16/Catalogs/dr16-line-sky-mask.txt --rebin 3 --mode spplate --lambda-min 3600.0 --lambda-max 7235.0 --lambda-rest-min 2900.0 --lambda-rest-max 3120.0 --nit 20 --bi-max 0. --npix-min 50

In [ ]:
$PICCA_BASE/bin/picca_deltas.py --in-dir $EBOSS_v5_13_0/ --drq $Reproduce_dr16/Catalogs/cat_for_clustering.fits.gz --out-dir $Reproduce_dr16/Delta/Delta_LYA/Delta/ --iter-out-prefix $Reproduce_dr16/Delta/Delta_LYA/Log/delta_attributes --log $Reproduce_dr16/Delta/Delta_LYA/Log/input.log --dust-map $Reproduce_dr16/Catalogs/cat_for_clustering.fits.gz --flux-calib $Reproduce_dr16/Delta/Delta_LYA/Log/delta_attributes.fits.gz --ivar-calib $Reproduce_dr16/Delta/Delta_LYA/Log/delta_attributes.fits.gz --mask-file $Reproduce_dr16/Catalogs/dr16-line-sky-mask.txt --dla-vac $Reproduce_dr16/Catalogs/cat_for_DLA.fits --rebin 3 --mode spplate --lambda-min 3600.0 --lambda-max 7235.0 --lambda-rest-min 1040.0 --lambda-rest-max 1200.0 --nit 20 --bi-max 0. --vlss-max 1.0 --optical-depth 0.0023 3.64 LYA --npix-min 50

In [ ]:
$PICCA_BASE/bin/picca_cf.py --in-dir $Reproduce_dr16/Delta/Delta_LYA/Delta/ --out $Reproduce_dr16/Correlations/cf_z_all_data.fits --nproc 32 --fid-Om 0.3147 --fid-Or 0.0

In [ ]:
$PICCA_BASE/bin/picca_dmat.py --in-dir $Reproduce_dr16/Delta/Delta_LYA/Delta/ --out $Reproduce_dr16/Correlations/dmat_z_all_data.fits --rej 0.99 --nproc 32 --fid-Om 0.3147 --fid-Or 0.0

In [ ]:
$PICCA_BASE/bin/picca_metal_dmat.py --in-dir $Reproduce_dr16/Delta/Delta_LYA/Delta/ --out $Reproduce_dr16/Correlations/metal_dmat_z_all_data.fits --nproc 32 --fid-Om 0.314569514863487 --fid-Or 7.97505418919554e-5 --rej 0.999 --abs-igm 'CIV(eff)' 'SiII(1260)' 'SiIII(1207)' 'SiII(1193)' 'SiII(1190)'

In [ ]:
$PICCA_BASE/bin/picca_export.py --data $Reproduce_dr16/Correlations/cf_z_all_data.fits --dmat $Reproduce_dr16/Correlations/dmat_z_all_data.fits --out $Reproduce_dr16/Correlations/cf_z_all_data-exp.fits.gz

In [ ]:
$PICCA_BASE/bin/picca_fitter2.py $Reproduce_dr16/Scripts/chi2_L0_10.ini

# People can also submit jobs using the scripts

In [ ]:
sbatch $Reproduce_dr16/Scripts/picca_deltas_calibration.batch
sbatch $Reproduce_dr16/Scripts/picca_deltas_calibration2.batch
sbatch $Reproduce_dr16/Scripts/picca_deltas_LYA.batch
sbatch $Reproduce_dr16/Scripts/cf_z.batch
sbatch $Reproduce_dr16/Scripts/dmat_z.batch
sbatch $Reproduce_dr16/Scripts/metal_dmat.batch
sbatch $Reproduce_dr16/Scripts/export.batch